In [1]:
# Create circuit to test transpiler on
from qiskit import QuantumCircuit, qpy
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import numpy as np
# Qiskit Runtime
from qiskit_aer import AerSimulator as Aer
from qiskit_ibm_runtime import QiskitRuntimeService, Batch,SamplerV2
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.metrics import mean_squared_error,root_mean_squared_error
from scipy import stats
import time
import logging
from datetime import datetime
from multiprocessing import Pool
import QRNN as QRNN
from qiskit.circuit.library import XGate, YGate
from qiskit.transpiler import InstructionProperties
from qiskit.transpiler import PassManager
from qiskit.circuit.equivalence_library import (
    SessionEquivalenceLibrary as sel,
)
from qiskit.transpiler.passes import BasisTranslator
from qiskit.transpiler.passes.scheduling import (
    ALAPScheduleAnalysis,
    PadDynamicalDecoupling,
)
date = str(datetime.now())

from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

In [2]:
n_qubits = 12 # must be even and less than 156. Going above 16 with low sparsity will increase depth of the circuit and transpile time
repeat_blocks = 3
ent_sparsity = 0.5 #These parameters can reduce the depth and number of SWAPs in the circuit by increasing them, but may hurt the performance.
mem_sparsity = 0.4 #Same as above
qc_name = 'ibm_marrakesh'
seed = 123123

#Shots
shots = 750 #If there is a buffer overflow, lowering this or increasing delay in circuit will help, but we usually need 30k shots minimum, so we have to run 40 of these or more

service = QiskitRuntimeService(channel="ibm_quantum",token='INSERT_TOKEN')

backend = service.backend(qc_name,use_fractional_gates=False)
target = backend.target
basis_gates = list(target.operation_names)
#backend= least_busy()
#backend = GenericBackendV2(num_qubits=50)
#backend = Aer.from_backend(backend)

# #Get data
start = 500 #points to ignore
n_pts = 2000 # points to use. there is only one circuit now.
train_input_signal, test_input_signal, train_data, test_data = QRNN.get_lorenz_data(startup=start,n_pts=n_pts)

# #Get circuit
qrnn_circuit, register_names = QRNN.QRNN(train_input_signal,n_qubits=n_qubits,context_length=3,repeat_blocks=repeat_blocks,ent_sparsity=ent_sparsity,mem_sparsity=mem_sparsity,seed=seed)


t_qc = generate_preset_pass_manager(optimization_level=3,backend=backend).run(qrnn_circuit)
"""
X = XGate()
Y = YGate()
#Create dynamical decoupling
dd_sequence = [X, Y, X, Y]

y_gate_properties = {}
for qubit in range(target.num_qubits):
    y_gate_properties.update(
        {
            (qubit,): InstructionProperties(
                duration=target["x"][(qubit,)].duration,
                error=target["x"][(qubit,)].error,
            )
        }
    )

target.add_instruction(YGate(), y_gate_properties)
t_qc = PassManager(
    [
        ALAPScheduleAnalysis(target=target),
        PadDynamicalDecoupling(target=target, dd_sequence=dd_sequence),
    ]
).run(t_qc)

t_qc = BasisTranslator(sel, basis_gates)(t_qc)"""

#t_qc = generate_preset_pass_manager(optimization_level=3,backend=backend).run(qrnn_circuit)
#sampler = SamplerV2(mode=backend, options={"experimental":{"execution":{"rep_delay":{.0005}}}})



[[ 0.00284433 -0.24469469]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.45766431 -0.05294298]
 [ 0.          0.92548639]
 [-0.6182406   0.        ]]
[-0.001434    0.         -0.14368148  0.02531314  0.         -0.0823018 ]
Depth of circuit: 
207454


In [78]:
# from qiskit import qpy
# We have a compiled circuit that works as well.
 
# with open("marrakesh_12qubit_500-2000data_rep3_sparsityent_.5_sparsitymem_.4_seed_123123_context_3_750_shots.qpy", "wb") as file:
#     qpy.dump(t_qc, file)

In [3]:
print('Depth of circuit: ')
print(t_qc.depth())

Depth of circuit: 
207454


In [4]:
t_qc.count_ops()

OrderedDict([('rz', 449851),
             ('sx', 425830),
             ('cz', 119796),
             ('x', 11988),
             ('measure', 11988),
             ('reset', 11988),
             ('barrier', 3996)])

In [5]:
#Get total number of operations
n_ops = 0
for instr, _, _ in t_qc.data:
    n_ops += 1
print('Number of operations: ')
print(n_ops)


/var/folders/d7/5zdtyjdj6bqf_gtsn20v944c0000gn/T/ipykernel_44717/1890759391.py:3: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for instr, _, _ in t_qc.data:


Number of operations: 
1035437


In [6]:
sampler = SamplerV2(mode=backend,options={"execution":{"rep_delay":0.0005}})#, options={"experimental":{"execution":{"rep_delay":{.0005}}}})

In [7]:
jobs = []

In [8]:
#Change this to get more than 1 job, but it is best to make sure circuit executes beforehand. 40 Jobs with 750 shots is 30k
total_jobs = 1
for i in range(total_jobs):
    jobs.append(sampler.run([t_qc], shots=shots))

In [13]:
for job in jobs:
    print(job.status()) 

ERROR


In [14]:
# for job in jobs:
#     print(job.error_message())

Job cy800hanrmz00085v5sg has failed:
Error code 8045; A buffer overflow occurred during result collection. Reduce the number of shots or measurements, or increase the rep_delay 


In [11]:
service.usage()

{'period': {'start': '2025-01-01T00:00:00.000Z',
  'end': '2025-01-31T23:59:59.999Z'},
 'byInstance': [{'instance': 'ibm-q/open/main',
   'quota': 600,
   'usage': 0,
   'pendingJobs': 0,
   'maxPendingJobs': 3}]}

In [35]:
jobs[23].error_message()

'Job cy58c0rnrmz00085ffk0 has failed:\nError code 1800; Unknown error.'

In [19]:
time = backend.dt * 6000
time


2.4e-05

In [9]:
backend.dt

4e-09

In [76]:
results = []
for job in jobs:
    results.append(job.result()[0].data)
#results2 = job2.result()
#results3 = job3.result()
# results4 = job4.result()
# results5 = job5.result()

In [77]:
#results_list = [results[0].data]#,results2[0].data,results3[0].data]
expectation = QRNN.get_probability_matrix(results, register_names, n_qubits//2)

np.save(f'expectations_{date}_{n_qubits}_{qc_name}_shots_60000_seed_{seed}_pts_{start}_{n_pts}_{test_pts}.npy',expectation)


In [12]:
#Run test job
#Get circuit
qrnn_circuit_test, test_register_names = QRNN.QRNN(test_input_signal,n_qubits=n_qubits,context_length=3,repeat_blocks=repeat_blocks,seed=seed)

t_qc_test = generate_preset_pass_manager(optimization_level=3,backend=backend).run(qrnn_circuit_test)


t_qc_test = PassManager(
    [
        ALAPScheduleAnalysis(target=target),
        PadDynamicalDecoupling(target=target, dd_sequence=dd_sequence),
    ]
).run(t_qc_test)

t_qc_test = BasisTranslator(sel, basis_gates)(t_qc_test)
print('Depth of circuit: ')
print(t_qc_test.depth())
#f.write(f'\n Depth of test circuit: {t_qc.depth()}')



[[ 0.          0.        ]
 [-1.33123903 -0.18202904]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [-0.59892589  0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.87878391]
 [ 0.         -0.48171322]
 [ 0.          0.        ]
 [ 0.          0.        ]]
Depth of circuit: 
60398


In [14]:
jobs_test = []
for i in range(total_jobs):
    jobs_test.append(sampler.run([t_qc_test], shots=shots))
# job_test = sampler.run([t_qc_test], shots=shots)
# #job_test2 = sampler.run([t_qc_test], shots=shots)
# #job_test3 = sampler.run([t_qc_test], shots=shots)
# # job_test4 = sampler.run([t_qc_test], shots=shots)
# # job_test5 = sampler.run([t_qc_test], shots=shots)

In [30]:
for i in range(total_jobs):
    print(jobs_test[i].status())
# print(job_test.status())
#print(job_test2.status())
#print(job_test3.status())
# print(job_test4.status())
# print(job_test5.status())

NameError: name 'jobs_test' is not defined

In [ ]:
results_test = []
for i in range(total_jobs):
    results_test.append(jobs_test[i].result()[0].data)
# results_test = job_test.result()
#results_test2 = job_test2.result()
#results_test3 = job_test3.result()
# results_test4 = job_test4.result()
# results_test5 = job_test5.result()

In [ ]:
#results_list_test = [results_test[0].data,results_test2[0].data,results_test3[0].data,results_test4[0].data,results_test5[0].data]

In [ ]:
#results_list_test = [results_test[0].data]#,results_test2[0].data,results_test3[0].data]
expectation_test = QRNN.get_probability_matrix(results_test, test_register_names, n_qubits//2)

np.save(f'./expectations_test_{date}_{n_qubits}_{qc_name}_shots_{shots}_seed_{seed}_{repeat_blocks}_{start}_{n_pts}_{test_pts}.npy',expectation_test)